In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [2]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo64'
DB_PASSWORD = 'grupo64'
DB_NAME = 'grupo64e2'

In [3]:
# Ruta del archivo CSV
archivo_csv = '../datos e2/restaurantes2.csv'

# Configuración de la conexión
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()

    # Dropear la tabla si ya existe
    drop_table_query = "DROP TABLE IF EXISTS restaurantes;"
    cursor.execute(drop_table_query)
    print("Tabla restaurantes eliminada si existía")

    # Crear la tabla (ajusta esto según tu esquema de datos)
    crear_tabla_query = """
    CREATE TABLE IF NOT EXISTS restaurantes (
        id SERIAL,
        nombre varchar   NOT NULL,
        vigente bool   NOT NULL,
        estilo varchar   NOT NULL,
        repartomin INT  NOT NULL,
        sucursal varchar NOT NULL,
        direccion varchar   NOT NULL,
        telefono varchar    NOT NULL,
        area varchar NOT NULL,
        CONSTRAINT pk_platoe PRIMARY KEY (
            id))
    """
    cursor.execute(crear_tabla_query)
    conexion.commit()

    # Preparar el query de inserción
    insercion_query = """
    INSERT INTO restaurantes (nombre, vigente, estilo, repartomin, sucursal, direccion, telefono, area)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """

    # Leer y procesar el archivo CSV
    with open(archivo_csv, 'r') as file:
        lista_restaurant = []
        restaurant_actual = ""
        next(file)

        for linea in file:
            restaurant_actual += linea.strip()
            valores = restaurant_actual.split(';')
            if len(valores) == 8:
                lista_restaurant.append(valores)
                restaurant_actual = ""
            elif len(valores) > 8:
                print(f"Línea malformada: {restaurant_actual}")
                restaurant_actual = ""

        for restaurant in lista_restaurant:
            try:
                cursor.execute(insercion_query, (restaurant[0], restaurant[1], restaurant[2], int(restaurant[3]), restaurant[4], restaurant[5], restaurant[6], restaurant[7]))
            except Error as e:
                    print(f"Error al insertar la línea {linea}: {e}")
                    conexion.rollback()
                    continue

    # Confirmar la inserción
    conexion.commit()

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

Tabla restaurantes eliminada si existía


In [4]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM restaurantes;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

(1, 'DCCRestaurant', True, 'fast food', 10000, 'Las Parcelas', 'Av. Las Parcelas 1234, Macul, Santiago', '562994152387', ' Macul')
(2, 'Meet Burger', False, 'fast food', 5000, 'Providencia', 'Av. Providencia 456, Las Condes, Santiago', '56871243659', ' Las Condes')
(3, 'Meet Burger', False, 'fast food', 5000, 'Vitacura', 'Av. Vitacura 789, Vitacura, Santiago', '56274568231', ' Vitacura')
(4, 'Meet Burger', False, 'fast food', 5000, 'Apoquindo', 'Av. Apoquindo 123, Las Condes, Santiago', '56321078546', ' Las Condes')
(5, 'Meet Burger', False, 'fast food', 5000, 'Kennedy', 'Av. Kennedy 456, Las Condes, Santiago', '562109876452', ' Las Condes')
(6, 'Burger Queen', False, 'fast food', 8000, 'Padre Mariano', 'Calle Padre Mariano 789, La Reina, Santiago', '56784561032', ' La Reina')
(7, 'Burger Queen', False, 'fast food', 8000, 'Larra’n', 'Av. Larra’n 123, La Reina, Santiago', '562367891254', ' La Reina')
(8, 'Delicia China', False, 'internacional', 11000, 'Recoleta', 'Av. Recoleta 456, Huec